In [1]:
import uproot
import pandas as pd
import awkward as ak
file_dir = "/home/JYChen/EnergyRegression/samples/DoubleElectron_FlatPT-1to500_13p6TeV/"
IdealPath = f"{file_dir}DoubleElectron_ECALIdealIC_PostEE_124X_30032023.root"
RealPath = f"{file_dir}DoubleElectron_ECALRealIC_PostEE_124X_02042023.root"
treename = ""

In [2]:
tree = uproot.open(IdealPath)["egRegTree"]

In [3]:
print(tree.keys())

['nrVert', 'rho', 'nrPUInt', 'nrPUIntTrue', 'evt', 'sc', 'ssFull', 'ssFrac', 'ele', 'pho', 'eleSSFull', 'phoSSFull', 'mc', 'clus1', 'clus2', 'clus3']


In [6]:
df = tree.arrays(['rho', 'sc', 'ssFull', 'ele', 'mc'], library="pd",)

In [7]:
df

,rho,sc,ssFull,ele,mc
0,4.195293,"{'rawEnergy': 812.9088745117188, 'rawESEnergy'...","{'e3x3': 779.7754516601562, 'e5x5': 815.754089...","{'et': 249.94017028808594, 'energy': 864.30664...","{'energy': 869.5516357421875, 'pt': 251.492156..."
1,4.195293,"{'rawEnergy': 851.2734985351562, 'rawESEnergy'...","{'e3x3': 830.8706665039062, 'e5x5': 854.211791...","{'et': 252.51243591308594, 'energy': 873.01422...","{'energy': 869.5516357421875, 'pt': 251.492156..."
2,17.619236,"{'rawEnergy': 256.80474853515625, 'rawESEnergy...","{'e3x3': 217.0644073486328, 'e5x5': 244.071716...","{'et': 158.70974731445312, 'energy': 290.47375...","{'energy': 294.678955078125, 'pt': 160.9854278..."
3,17.619236,"{'rawEnergy': 282.0928649902344, 'rawESEnergy'...","{'e3x3': 246.0587921142578, 'e5x5': 275.997314...","{'et': 160.4888153076172, 'energy': 293.774658...","{'energy': 294.678955078125, 'pt': 160.9854278..."
4,29.529261,"{'rawEnergy': 37.29194641113281, 'rawESEnergy'...","{'e3x3': 19.886667251586914, 'e5x5': 24.351324...","{'et': 23.511146545410156, 'energy': 45.295555...","{'energy': 48.09366989135742, 'pt': 24.9610919..."
...,...,...,...,...,...
34919165,31.622850,"{'rawEnergy': 368.8412170410156, 'rawESEnergy'...","{'e3x3': 353.5511779785156, 'e5x5': 368.024230...","{'et': 271.7292785644531, 'energy': 375.274291...","{'energy': 379.8775634765625, 'pt': 275.075286..."
34919166,14.455096,"{'rawEnergy': 2068.220703125, 'rawESEnergy': 1...","{'e3x3': 2020.287841796875, 'e5x5': 2084.18090...","{'et': 402.0955810546875, 'energy': 2141.16137...","{'energy': 2152.004638671875, 'pt': 404.184661..."
34919167,14.455096,"{'rawEnergy': 2096.3505859375, 'rawESEnergy': ...","{'e3x3': 2058.515380859375, 'e5x5': 2111.11621...","{'et': 404.94915771484375, 'energy': 2154.5639...","{'energy': 2152.004638671875, 'pt': 404.184661..."
34919168,44.513596,"{'rawEnergy': 38.77729797363281, 'rawESEnergy'...","{'e3x3': 38.37193298339844, 'e5x5': 40.2717208...","{'et': 37.71287155151367, 'energy': 40.5661125...","{'energy': 40.21603012084961, 'pt': 37.3829269..."


In [12]:
def trans_to_df(ttree, axis=1, debug=False):
    branches =  ttree.keys()
    if debug:
        df = ttree.arrays(branches, library="pd", entry_stop=1000)
    else:
        df = ttree.arrays(branches, library="pd",)
    dfs = []
    for branch in branches:
        obj = eval(f"ak.to_dataframe(df.{branch})")
        obj = obj.rename(columns={oldname: f"{branch}_{oldname}" for oldname in obj.columns})
        dfs.append(obj)
    return pd.concat(dfs, axis=axis)

In [13]:
df = trans_to_df(tree, axis=1, debug=True)

In [27]:
df = tree.arrays(tree.keys(), library="pd", entry_stop=1000)

pandas.core.series.Series

In [20]:
train_dataset = df.sample(frac=0.8, random_state=0)

In [22]:
train_dataset.index

Index([993, 859, 298, 553, 672, 971,  27, 231, 306, 706,
       ...
       507, 228, 673, 595, 445, 117, 464,  25, 110, 149],
      dtype='int64', name='entry', length=800)

In [4]:
import json
with open("/home/JYChen/EnergyRegression/config/Run3Electron_PostEE.json") as file:
    conf = json.load(file)

In [7]:
def setup_dataset(df, vars):
    operator = ["+", "-", "*", "/"]
    

'Ideadf.mc_energy/Ideadf.sc_rawEnergy'

In [2]:
import pathlib
import shutil
logdir = pathlib.Path("log/tfTrainer")/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)